In [ ]:
import pandas as pd
import datetime as dt
import yfinance as yf
import json
from stockstats import wrap 
import os



# get watch_list

In [ ]:
with open('watch_list.json') as f:
    watch_list = json.load(f)

# fetch data from yfinance api

In [ ]:
#create stock_data directory if it doesn't exist
if not os.path.exists('stock_data'):
    os.makedirs('stock_data')
for symbol in watch_list:
    tk = yf.Ticker(f"{symbol}.BK")
    df = tk.history(period='3mo', interval='1d')
    df.to_csv(f"stock_data/{symbol}.csv")




# apply indicator to data

In [ ]:
#create stock_data_with_indicator directory if it doesn't exist
if not os.path.exists('stock_data_with_indicator'):
    os.makedirs('stock_data_with_indicator')
for symbol in watch_list:
    df = pd.read_csv(f"stock_data\{symbol}.csv")
    df_ta = wrap(df)
    df_ta.init_all()
    df.to_csv(f"stock_data_with_indicator\{symbol}.csv")

# show result with plotly and save result to file

In [ ]:
import random
import plotly.graph_objects as go
from tradingview_ta import TA_Handler, Interval
import tradingview_ta

symbol = random.choice(watch_list)
handler = TA_Handler(
    symbol=symbol,
    exchange="SET",
    screener="thailand",
    interval=Interval.INTERVAL_1_DAY,
    timeout=None
)
df = pd.read_csv(f"stock_data_with_indicator\{symbol}.csv")
df.set_index('date', inplace=True)
fig = go.Figure(data=[go.Candlestick(x=df.index,
                                     open=df['open'],
                                     high=df['high'],
                                     low=df['low'],
                                     close=df['close'],
                                     hoverlabel=dict(bgcolor='white'),
                                     name=symbol
                                        )])


# fig.add_trace(go.Scatter(
#     x=df.index, y=df["middle"], line_shape='spline', name="Middle",))
fig.add_trace(go.Scatter(
    x=df.index, y=df["close_10_sma"], line_shape='spline', name="close_10_sma",))
fig.add_trace(go.Scatter(
    x=df.index, y=df["close_50_sma"], line_shape='spline', name="close_50_sma",))

# fig name
fig.update_layout(
    title=f"#{symbol} [last update {dt.datetime.now().strftime('%y/%d/%m %H:%M')}]<br>{' || '.join([f'{k} : {v}' for k,v in handler.get_analysis().summary.items()])}<br>[ open = {df['open'].iloc[-1]:.2f} || high = {df['high'].iloc[-1]:.2f} || low = {df['low'].iloc[-1]:.2f} || close = {df['close'].iloc[-1]:.2f} ]")
# hide bottom range slider
fig.update_layout(xaxis_rangeslider_visible=False)

# removing all empty dates
dt_all = pd.date_range(start=df.index[0],end=df.index[-1])
dt_obs = [d.strftime("%Y-%m-%d") for d in pd.to_datetime(df.index)]
dt_breaks = [d for d in dt_all.strftime("%Y-%m-%d").tolist() if not d in dt_obs]
fig.update_xaxes(rangebreaks=[dict(values=dt_breaks)])

if not os.path.exists("images"):
    os.mkdir("images")

fig.write_image(f"images/{symbol}-{dt.date.today()}.webp",format="webp",scale=4)
fig.show()
